In [2]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [4]:
import requests
from bs4 import BeautifulSoup
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import csv
import smtplib, ssl
import urllib


In [5]:
def getPageContent(url):
  page = requests.get(url)
  pagesoup = BeautifulSoup(page.content, features="lxml")
  return pagesoup

In [7]:
playerNames = {}
chancesOfScoring = []
teamChances = {}
pageContent = getPageContent("https://apuestas.kirolbet.es/esp/Sport/Competicion/1")
betsContainer = pageContent.find("div", {"id": "diveve1"})
betsContainerL1 = betsContainer.find("ul", {"class": "apuestas"})
for ligaEventsContainer in betsContainerL1.find_all("li", {"class": "filtroCategoria filtroCat-40"}):
    matchContainer = ligaEventsContainer.find("div", {"class": "infoEve"})
    matchContainerL1 = matchContainer.find("div", {"class": "info"})
    matchContainerL2 = matchContainerL1.find("span", {"class": "partido"})
    matchLinks = matchContainerL2.find_all("a")
    matchLink = matchLinks[1]["href"]
    matchInfoSoup = getPageContent("https://apuestas.kirolbet.es" + str(matchLink))
    quotesContainer = matchInfoSoup.find("div", {"class": "pruebaToggle"})
    teamNamesContainer = quotesContainer.find("div", {"class": "graphite_partido ksIcon"})
    teamNamesContainerL1 = teamNamesContainer.find("h3")
    teamNamesWhole = teamNamesContainerL1.find('span').get_text()
    teamName1 = teamNamesWhole[0: (teamNamesWhole.find('v')-1)]
    teamName2 = teamNamesWhole[(teamNamesWhole.find('.')+1):]
    quotesList = quotesContainer.find("div", {"class": "prox_eventos"})
    chanceOfWinningContainer = quotesList.find("ul", {"market-group-id": "market.MarketGroupId"})
    chanceOfWinningContainerL1 =  chanceOfWinningContainer.find("li", {"class": "ksToggle"})
    chanceOfWinningContainerL2 = chanceOfWinningContainerL1.find("div")
    chanceOfWinningContainerL3 = chanceOfWinningContainerL2.find("ul")
    chanceOfDrawing = 0
    for quotes in chanceOfWinningContainerL3.find_all("li"):
        quoteContainer = quotes.find("a")
        quoteContainerL1 = quotes.find("p")
        chanceOfWinningAndTeamNumber = quoteContainerL1.find_all("span")
        teamNumber = chanceOfWinningAndTeamNumber[0].get_text()
        chanceOfWinningDrawingOrLosing = float(chanceOfWinningAndTeamNumber[1].get_text().replace(",", "."))
        if(teamNumber == "X"):
            chanceOfDrawing = chanceOfWinningDrawingOrLosing
        else:
            teamChances[teamNumber] = chanceOfWinningDrawingOrLosing
    if (quotesList.find("ul", {"id": "ulPlayers"}) != None):
        playerListContainer = quotesList.find("ul", {"id": "ulPlayers"})
        playerListContainerL1 = playerListContainer.find("div", {"class":"participante_col"})
        playerListContainerL2 = playerListContainer.find("div", {"class": "valores"})
        playerList = playerListContainerL2.find("ul")
        for player in playerList.find_all("li"):
            playerNameContainer = player.find("p", {"class": "tipe1"})
            playerSpans = player.find_all("span")
            playerTeamNumber = playerSpans[0].get('class')
            playerTeamNumberText = playerTeamNumber[0][-1]
            playerName = playerSpans[1].get_text()
            playerNames[playerName] = playerTeamNumberText
            playerNames[playerName] = [playerNames[playerName], teamChances[playerNames[playerName]]]
            if (playerTeamNumberText == '1'):
                playerTeamName = teamName1
            else:
                playerTeamName = teamName2
            playerNames[playerName][0] = playerTeamName
        chanceOfScoringContainer = playerListContainer.find("ul", {"des": "Goleador"})
        chanceOfScoringContainerL1 = chanceOfScoringContainer.find("li")
        chanceOfScoringContainerL2 = chanceOfScoringContainerL1.find("div", {"class": "valores"})
        chanceOfScoringList = chanceOfScoringContainerL2.find("ul")
        for chance in chanceOfScoringList.find_all("li"):
            chanceOfScoringContainerL3 = chance.find("a")
            chanceOfScoringContainerL4 = chanceOfScoringContainerL3.find("span")
            chanceOfScoring = 100/float(chanceOfScoringContainerL4.get_text().replace(",", "."))
            chancesOfScoring.append(chanceOfScoring)
        for k, v in playerNames.items():
            index = list(playerNames.keys()).index(k)
            playerNames[k].append(chancesOfScoring[index])
playersDataFrame = pd.DataFrame.from_dict(playerNames, orient='index')
cleanDataFrame = playersDataFrame.iloc[:,:3]
cleanDataFrame = cleanDataFrame.rename(columns={0: 'TeamName', 1: 'ChanceOfWinning', 2: 'ChanceOfScoring'})
cleanDataFrame

,TeamName,ChanceOfWinning,ChanceOfScoring
"Romero, Isaac",Sevilla,2.45,30.769231
"Veliz, Alejo",RCD Espanyol,2.95,28.571429
"Cardona, Irvin",RCD Espanyol,2.95,28.571429
"Puado, Javi",RCD Espanyol,2.95,28.571429
"Cheddira, Walid",RCD Espanyol,2.95,28.571429
...,...,...,...
"Odriozola, Álvaro",Real Sociedad,1.75,4.000000
"Cruz, Juan <osa>",Osasuna,4.80,4.000000
"Vidal, Nacho",Osasuna,4.80,4.000000
"Catena, Alejandro",Osasuna,4.80,4.000000


In [9]:
getPageContent("https://www.goldenbull.es/betting#sports-hub/football/spain/la_liga_2")

<!DOCTYPE html>
<html lang="es" translate="no"><head><meta charset="utf-8"/><meta content="width=device-width" name="viewport"/><link href="https://www.goldenbull.es/betting" rel="canonical"/><title>Betting | Online Casino | GoldenBull.es</title><meta content="Tenemos las últimas cuotas y mercados en vivo para todos tus deportes favoritos. Más de 5.000 eventos diarios en los que apostar" name="description"/><meta content="5" name="next-head-count"/><meta content="telephone=no" name="format-detection"/><meta content="width=device-width, initial-scale=1.0" name="viewport"/><meta content="#05161D" name="msapplication-TileColor"/><meta content="#05161D" name="theme-color"/><meta content="yes" name="apple-mobile-web-app-capable"/><meta content="yes" name="mobile-web-app-capable"/><link crossorigin="use-credentials" href="/manifest.json" rel="manifest"/><link href="/_next/static/media/apple-touch-icon.2a81cdbb.png" rel="apple-touch-icon" sizes="180x180"/><link href="/_next/static/media/apple